In [4]:
import pandas as pd
from retrieval.models.basemodels import TfIdf
import numpy as np
import swifter
import os
import pandas as pd

def positionPidQid(best_ind_in_m, pid, qid):
    return best_ind_in_m[qid,pid]

def passageFromTriple(triple, df_passages, df_queries):
    passages = df_passages.iloc[triple[1:]]['passage'].values
    print(passages)
    tf_idf = TfIdf(passages)
    k = len(passages)
    queries = [df_queries.iloc[triple[0]]['query']]
    print('-----')
    print(queries)

    best_ind_all = tf_idf.answerQuestion(queries,k)
    best_ind_in_m = np.argsort(best_ind_all, axis=1)

    print(best_ind_in_m.shape)
    print(best_ind_in_m[0,0])
    return best_ind_in_m[0,0]


def statistics(path, passages_from_triples=False):
    print(path)

    df_passages = pd.read_csv(path + '/passages.tsv', sep ='\t')
    df_queries = pd.read_csv(path + '/queries.tsv', sep ='\t')
    df_triple = pd.read_csv(path + '/triples.tsv', sep ='\t')

    if passages_from_triples:
        df_triple['pos+10'] = df_triple.swifter.apply(lambda row : passageFromTriple(row, df_passages, df_queries), axis=1)

    else:
        passages = df_passages.passage.values
        queries = df_queries['query'].values
        tf_idf = TfIdf(passages)
        k = len(passages)
        best_ind_all = tf_idf.answerQuestion(queries,k)
        best_ind_in_m = np.argsort(best_ind_all, axis=1)
        
        df_triple['pos+'] = df_triple.swifter.apply(lambda row : positionPidQid(best_ind_in_m, row["PID"], row["QID+"]), axis=1)
        df_triple['pos-'] = df_triple.swifter.apply(lambda row : positionPidQid(best_ind_in_m, row["PID"], row["QID-"]), axis=1)

        mrrplus = df_triple['pos+'].apply(lambda x: 1.0/(x+1)).sum() / len(df_triple)
        mrrminus = df_triple['pos-'].apply(lambda x: 1.0/(x+1)).sum() / len(df_triple)
        
        

        print('describe pos+')
        print(f'MRR pos+: {mrrplus}')
        print(df_triple['pos+'].describe())

        print('describe pos-')
        print(f'MRR pos-: {mrrminus}')
        print(df_triple['pos+'].describe())

        print('----------------------------')



In [5]:


SEARCH_PATH = "../../data/"

for root, dirs, files in os.walk(SEARCH_PATH):
    
    # is a target folder
    if 'passages.tsv' in files and ('MS' in root ):
        continue
        print(statistics(root, passages_from_triples=True))
    elif 'passages.tsv' in files and ('dc' in root ):
        statistics(root, passages_from_triples=False)

../../data/fandoms_qa/dc_comics/test
describe pos+
MRR pos+: 0.38779377263601367
count    74.000000
mean      6.945946
std       8.608521
min       0.000000
25%       1.000000
50%       3.000000
75%      10.750000
max      36.000000
Name: pos+, dtype: float64
describe pos-
MRR pos-: 0.317772743535755
count    74.000000
mean      6.945946
std       8.608521
min       0.000000
25%       1.000000
50%       3.000000
75%      10.750000
max      36.000000
Name: pos+, dtype: float64
----------------------------
../../data/fandoms_qa/dc_comics/all
describe pos+
MRR pos+: 0.5736752679403928
count    730.000000
mean      22.950685
std       65.150730
min        0.000000
25%        0.000000
50%        1.000000
75%        6.000000
max      363.000000
Name: pos+, dtype: float64
describe pos-
MRR pos-: 0.4260844649667486
count    730.000000
mean      22.950685
std       65.150730
min        0.000000
25%        0.000000
50%        1.000000
75%        6.000000
max      363.000000
Name: pos+, dtype: fl

In [13]:
file = '../../../data/MSMARCO/ms_marco_v1.1/train/triples.tsv'
df = pd.read_csv(file, sep ='\t')
df
df_passages.iloc[triple['QID']].values()[0]

,QID,PID+,PID-,PID-.1,PID-.2,PID-.3,PID-.4,PID-.5,PID-.6,PID-.7,PID-.8
0,0,5,0,1,2,3,4,6,7,8,9
1,1,11,10,12,13,14,15,16,10,12,10
2,2,21,17,18,19,20,22,23,24,25,26
3,3,35,27,28,29,30,31,32,33,34,34
4,4,38,36,37,39,40,41,42,43,42,36
...,...,...,...,...,...,...,...,...,...,...,...
88509,82321,626876,626877,626878,626877,626877,626878,626877,626878,626877,626878
88510,82322,626879,215069,215067,186472,416252,626880,626881,626882,626883,215069
88511,82323,626885,626884,626886,626887,626888,626889,626887,626886,626887,626889
88512,82324,626892,626890,626891,42146,626893,626894,626895,626896,626897,626897
